In [2]:
from langchain.tools import Tool

# Define a tool
def search_tool(query: str) -> str:
    return f"Search results for: {query}"

tools = [
    Tool(
        name="Search",
        func=search_tool,
        description="Useful for searching the web or answering questions about current events."
    )
]

In [3]:
from langchain.prompts import PromptTemplate

prompt = PromptTemplate(
    input_variables=["input", "chat_history", "agent_scratchpad"],
    template="""
    You are a helpful assistant. Use tools when necessary to answer the following question:
    Input: {input}
    Chat History: {chat_history}
    Agent Scratchpad: {agent_scratchpad}
    """
)

In [4]:
input_data = {
    "input": "What is 10+10?",
    "chat_history": [],
    "agent_scratchpad": ""
}

In [5]:
from langchain_ollama.chat_models import ChatOllama

model_name = "llama3.2:1b-instruct-fp16"

# initialize one LLM with temperature 0.0, this makes the LLM more deterministic
llm = ChatOllama(temperature=0.0, model=model_name)

In [7]:
# Define the agent pipeline
agent = (
    {
        "input": lambda x: x["input"],
        "chat_history": lambda x: x["chat_history"],
        "agent_scratchpad": lambda x: x.get("agent_scratchpad", "")
    }
    | prompt
    | llm
)

# Run the agent
response = agent.invoke(input_data)
print(response.tool_calls)

[]


In [52]:
# bound_llm = llm.bind_tools(tools, tool_choice="Search")

In [ ]:
from langchain.agents import initialize_agent
from langchain.tools import Tool
from langchain_ollama.chat_models import ChatOllama
from langchain.memory import ConversationBufferMemory

model_name = "llama3.2:1b-instruct-fp16"

# Define a tool
def search_tool(query: str) -> str:
    return f"Search results for: {query}"

tools = [
    Tool(
        name="Search",
        func=search_tool,
        description="Useful for searching the web or answering questions about current events."
    )
]

# Initialize the LLM
llm = ChatOllama(temperature=0.0, model=model_name)

# Initialize memory
memory = ConversationBufferMemory()

# Create the agent with memory
agent = initialize_agent(
    tools,  # List of tools
    llm,  # Language model
    agent="zero-shot-react-description",  # Agent type
    verbose=True,  # Show detailed logs
    memory=memory,  # Add memory
    handle_parsing_errors=True  # Handle parsing errors
)

# Prepare input data
input_data = {
    "input": "What is 10+10?",
    "chat_history": memory.buffer  # Include chat history from memory
}

# Run the agent
response = agent.invoke(input_data)
print(response)

# Add the current interaction to memory
memory.save_context(
    {"input": input_data["input"]},
    {"output": response["output"]}
)

# Follow-up input
input_data = {
    "input": "Can you explain it in more detail?",
    "chat_history": memory.buffer  # Updated chat history
}

# Run the agent again
response = agent.invoke(input_data)
print(response)

# Add the follow-up interaction to memory
memory.save_context(
    {"input": input_data["input"]},
    {"output": response["output"]}
)